# Mono-S2S: Getting Started

This notebook explains how to run and extend the monotonic sequence-to-sequence experiments on HPC.

## 1. Project Overview

**Goal:** Compare adversarial robustness of monotonic vs. standard T5 models on summarization.

**Key idea:** Enforce non-negative weights ($W \geq 0$) in FFN layers via softplus reparameterization.

**Pipeline stages:**
| Stage | Script | Purpose |
|-------|--------|----------|
| 0 | `stage_0_setup.py` | Environment setup |
| 1 | `stage_1_prepare_data.py` | Download & preprocess datasets |
| 2 | `stage_2_train_baseline.py` | Train unconstrained T5 |
| 3 | `stage_3_train_monotonic.py` | Train monotonic T5 (W≥0) |
| 4 | `stage_4_evaluate.py` | ROUGE evaluation |
| 5 | `stage_5_uat_attacks.py` | Universal adversarial triggers |
| 6 | `stage_6_hotflip_attacks.py` | Gradient-based attacks |
| 7 | `stage_7_aggregate.py` | Compile final results |

## 2. Directory Structure

```
mono-s2s/
├── hpc_version/
│   ├── configs/
│   │   └── experiment_config.py    # All hyperparameters here
│   ├── scripts/
│   │   └── stage_*.py              # Pipeline stages
│   ├── jobs/
│   │   └── job_*.sh                # SLURM job scripts
│   ├── utils/
│   │   └── common_utils.py         # Shared utilities
│   ├── run_all.sh                  # Master orchestrator
│   ├── clean_all.sh                # Reset everything
│   └── clean_checkpoints.sh        # Reset models only
└── documentation/
    └── this notebook
```

## 3. Running an Experiment

### On Alpine HPC:

```bash
# Clone and navigate
git clone <repo_url>
cd mono-s2s/hpc_version

# Run full pipeline (submits SLURM jobs with dependencies)
./run_all.sh 42  # 42 is the random seed

# Monitor progress
squeue -u $USER
```

### Restarting:

```bash
# Clean everything and start fresh
./clean_all.sh
./run_all.sh 42

# Or just retrain models (keep datasets)
./clean_checkpoints.sh
./run_all.sh 42
```

## 4. Configuration

All hyperparameters are in `configs/experiment_config.py`. Key settings:

In [ ]:
# Key configuration options (from experiment_config.py)

# Model
MODEL_NAME = "t5-small"  # Options: t5-small, t5-base, t5-large

# Training
LEARNING_RATE = 5e-5
NUM_EPOCHS = 5           # Baseline epochs
MONOTONIC_NUM_EPOCHS = 7 # Monotonic gets more epochs
BATCH_SIZE = 4

# Evaluation
USE_FULL_TEST_SETS = False  # True for paper, False for quick tests
QUICK_TEST_SIZE = 200       # Samples when USE_FULL_TEST_SETS=False

# Attacks
ATTACK_TRIGGER_LENGTH = 5
ATTACK_NUM_ITERATIONS = 50

## 5. Adding a New Experiment

### Option A: Change hyperparameters

1. Edit `configs/experiment_config.py`
2. Run `./clean_checkpoints.sh` (keeps datasets)
3. Run `./run_all.sh <new_seed>`

### Option B: Add a new attack type

1. Create `scripts/stage_X_new_attack.py` following existing patterns
2. Create `jobs/job_X_new_attack.sh` (copy from job_5 or job_6)
3. Add to `run_all.sh` with appropriate dependencies

### Option C: Add a new model variant

1. Add constraint logic in `utils/common_utils.py` (see `make_model_monotonic()`)
2. Create new training script (copy from stage_3)
3. Update evaluation to include new model

## 6. Understanding the Monotonic Constraint

The key modification is in `utils/common_utils.py`:

In [ ]:
# Softplus parametrization ensures W >= 0
class SoftplusParametrization(nn.Module):
    def forward(self, V):
        return F.softplus(V)  # W = log(1 + exp(V)) >= 0

# Applied to FFN layers: wi, wi_0, wi_1, wo
# Attention, LayerNorm, residuals are UNCONSTRAINED

## 7. Output Files

Results are saved to `$SCRATCH/mono_s2s_results/`:

| File | Contents |
|------|----------|
| `evaluation_results.json` | ROUGE scores with confidence intervals |
| `uat_results.json` | Universal trigger attack results |
| `hotflip_results.json` | Gradient attack results |
| `final_results.json` | Aggregated summary |
| `experiment_summary.txt` | Human-readable report |

## 8. Troubleshooting

| Issue | Solution |
|-------|----------|
| Job stuck in queue | Normal, can take 24h+ on busy clusters |
| "Flag not created" error | Script auto-fixes if checkpoints exist |
| Out of memory | Reduce `BATCH_SIZE` in config |
| Want to resume | Just re-run `./run_all.sh`, it skips completed stages |

Check job logs:
```bash
tail -f logs/job_3_monotonic_*.out  # Live output
cat logs/job_3_monotonic_*.err      # Errors
```

## 9. Quick Reference

```bash
# Run experiment
./run_all.sh 42

# Check status
squeue -u $USER

# Clean and restart
./clean_all.sh && ./run_all.sh 42

# Retrain models only
./clean_checkpoints.sh && ./run_all.sh 42

# View results
cat $SCRATCH/mono_s2s_results/experiment_summary.txt
```